# Exemplo de ETL Simples

## Setup inicial

A idéia deste exercício é mostrar na prática como funciona um ETL simples para gerar um relatório em formato CSV de quantidade de emprestimos de veículos por região.
Os dados de fonte são dois arquivos: Um arquivo CSV com informações de empréstimos de filmes e clientes e outro arquivo CSV com informações sobre cidades.

Instalando a biblioteca Pandas

In [ ]:
!pip install pandas

Importando bibliotecas de manipulação de dados 

In [2]:
import pandas as pd

## Baixando os dados dos arquivos (Extract)

Para este caso como vamos lidar apenas com arquivos CSV não precisamos de mais nenhuma biblioteca além do pandas.

Carregando as informações dos arquivos (cada um para um dataframe diferente)...

In [3]:
dataframe_cidades = pd.read_csv('cidades_etl_example.csv')
dataframe_emprestimos = pd.read_csv('emprestimos_etl_example.csv')

In [4]:
#listando o top 5 para ver como é a tabela de regiões
dataframe_cidades.head(5)

,city_id,city,country
0,1,A Corua (La Corua),Spain
1,2,Abha,Saudi Arabia
2,3,Abu Dhabi,United Arab Emirates
3,4,Acua,Mexico
4,5,Adana,Turkey


In [5]:
#listando o top 5 para ver como é a tabela de pedidos
dataframe_emprestimos.head(5)

,rental_id,cust_name,city_id,rental_date,return_date
0,2,Tommy Collazo,423,2005-05-24 22:54:33,2005-05-28 19:40:33
1,3,Manuel Murrell,230,2005-05-24 23:03:39,2005-06-01 22:12:39
2,4,Andrew Purdy,48,2005-05-24 23:04:41,2005-06-03 01:43:41
3,5,Delores Hansen,235,2005-05-24 23:05:21,2005-06-02 04:33:21
4,6,Nelson Christenson,459,2005-05-24 23:08:07,2005-05-27 01:32:07


## Transformando os dados (Transform)
Neste passo estamos consolidando as duas informações utilizando a coluna city_id como chave

In [6]:
dataframe_join = dataframe_emprestimos.join(
    dataframe_cidades, 
    on='city_id',    #chave da junção entre as tabelas
    how='inner',     #tipo da junção, neste caso inner
    rsuffix='_geo',  #alias para campos com mesmo nome do lado direito (regioes)
    lsuffix='_rent'  #alias para campos com mesmo nome do lado esquerdo (pedidos)
)

In [7]:
#listando o top 5 para ver como ficou a tabela depois da junção
dataframe_join.head(5)

,city_id,rental_id,cust_name,city_id_rent,rental_date,return_date,city_id_geo,city,country
0,423,2,Tommy Collazo,423,2005-05-24 22:54:33,2005-05-28 19:40:33,424,Quilmes,Argentina
1873,423,1876,Tommy Collazo,423,2005-06-17 02:50:51,2005-06-18 07:21:51,424,Quilmes,Argentina
1974,423,1977,Tommy Collazo,423,2005-06-17 09:38:22,2005-06-23 14:19:22,424,Quilmes,Argentina
2072,423,2075,Tommy Collazo,423,2005-06-17 16:40:33,2005-06-19 13:55:33,424,Quilmes,Argentina
2895,423,2899,Tommy Collazo,423,2005-06-20 02:39:21,2005-06-23 06:47:21,424,Quilmes,Argentina


E agora estamos contando a quantidade de alugueis de filmes agrupado por país e cidade

In [8]:
#groupby vai agrupar o dataframe por país e cidade e count() vai contar todas as colunas, por 
#isso temos que adicionar ['rental_id'] para contar somente nessa coluna.
count_by_city = dataframe_join.groupby(['country', 'city'])['rental_id'].count()

In [9]:
#listando o top 5 para ver como é a tabela final
count_by_city.head(5)

country         city  
Afghanistan     Kabul     25
Algeria         Batna     33
                Bchar     25
                Skikda    26
American Samoa  Tafuna    26
Name: rental_id, dtype: int64

## Carregando os dados no destino (Load)
Neste caso será um csv também na mesma pasta somente para fins didáticos, mas pode ser um banco de dados, um arquivo JSON ou qualquer outro formato.

In [10]:
count_by_city.to_csv('./count_by_city_export.csv')